# Notebook 04: Data Modeling (Multi-Model Comparison)

**Mục tiêu:** Huấn luyện và so sánh các mô hình Binary Classification để tìm ra model tốt nhất.

**Các mô hình sử dụng:**
1. **XGBoost:** Gradient Boosting (Mạnh mẽ, chuẩn công nghiệp).
2. **Random Forest:** Bagging (Ổn định, ít bị overfitting).
3. **Logistic Regression:** Linear (Đơn giản, tốc độ cao, làm baseline).

**Input:** `train_features.pkl`
**Output:** `best_model.pkl` (Model tốt nhất được chọn tự động).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

# Import các mô hình
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# --- CẤU HÌNH ---
INPUT_FILE = '../../dataset_final/clean_data/train_features.pkl'
OUTPUT_DIR = '../../dataset_final/models'
os.makedirs(OUTPUT_DIR, exist_ok=True)
MODEL_FILE = os.path.join(OUTPUT_DIR, 'best_matcher.pkl')
FEATURE_LIST_FILE = os.path.join(OUTPUT_DIR, 'feature_names.pkl')
RANDOM_SEED = 42

## 1. Load và Chuẩn bị Dữ liệu

In [ ]:
if not os.path.exists(INPUT_FILE):
    raise FileNotFoundError(f"❌ Không tìm thấy file: {INPUT_FILE}")

df = pd.read_pickle(INPUT_FILE)
print(f"Tổng số mẫu: {len(df)}")

feature_cols = [c for c in df.columns if c.startswith('feat_')]
target_col = 'label'

X = df[feature_cols]
y = df[target_col]

# Chia tập dữ liệu
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y
)

print(f"Train shape: {X_train.shape} | Val shape: {X_val.shape}")

# Tính tỷ lệ mất cân bằng (dùng cho XGBoost & Class Weight)
ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)
print(f"Imbalance Ratio (Neg/Pos): {ratio:.2f}")

## 2. Huấn luyện Đa Mô hình
Chúng ta sẽ lần lượt train 3 model.

In [ ]:
models = {}

# --- MODEL 1: XGBOOST ---
print("\n🚀 Training XGBoost...")
xgb = XGBClassifier(
    n_estimators=200, learning_rate=0.05, max_depth=6,
    subsample=0.8, colsample_bytree=0.8,
    scale_pos_weight=ratio, # Cân bằng mẫu
    random_state=RANDOM_SEED,
    eval_metric='logloss',
    use_label_encoder=False,
    early_stopping_rounds=20 # Dừng sớm
)
xgb.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=0)
models['XGBoost'] = xgb

# --- MODEL 2: RANDOM FOREST ---
print("🚀 Training Random Forest...")
# Random Forest cần class_weight='balanced' để xử lý mất cân bằng
rf = RandomForestClassifier(
    n_estimators=200, max_depth=10,
    class_weight='balanced',
    random_state=RANDOM_SEED,
    n_jobs=-1 # Chạy đa luồng
)
rf.fit(X_train, y_train)
models['Random Forest'] = rf

# --- MODEL 3: LOGISTIC REGRESSION ---
print("🚀 Training Logistic Regression...")
# Logistic Regression cần Scale dữ liệu (StandardScaler) để hoạt động tốt
lr = make_pipeline(
    StandardScaler(),
    LogisticRegression(class_weight='balanced', max_iter=1000, random_state=RANDOM_SEED)
)
lr.fit(X_train, y_train)
models['Logistic Regression'] = lr

print("✅ Đã huấn luyện xong 3 mô hình!")

## 3. So sánh & Đánh giá (Evaluation)
Vẽ ROC Curve để xem model nào "bá đạo" nhất.

In [ ]:
plt.figure(figsize=(10, 8))

best_score = 0
best_model_name = ""
best_model_obj = None

print("=== KẾT QUẢ ROC-AUC ===")

for name, model in models.items():
    # Dự đoán xác suất
    if hasattr(model, "predict_proba"):
        y_prob = model.predict_proba(X_val)[:, 1]
    else:
        y_prob = model.decision_function(X_val)
        
    # Tính AUC
    auc = roc_auc_score(y_val, y_prob)
    print(f"{name}: {auc:.4f}")
    
    # Vẽ ROC
    fpr, tpr, _ = roc_curve(y_val, y_prob)
    plt.plot(fpr, tpr, lw=2, label=f"{name} (AUC = {auc:.4f})")
    
    # Lưu model tốt nhất
    if auc > best_score:
        best_score = auc
        best_model_name = name
        best_model_obj = model

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('So sánh ROC Curve giữa các mô hình')
plt.legend(loc="lower right")
plt.show()

## 4. Lưu Model tốt nhất
Script sẽ tự động chọn model có AUC cao nhất để lưu.

In [ ]:
print(f"\n🏆 Model chiến thắng: {best_model_name} với AUC = {best_score:.4f}")

# 1. Lưu Model
joblib.dump(best_model_obj, MODEL_FILE)

# 2. Lưu Feature List (Nếu model là Pipeline LR, ta vẫn cần list feature gốc)
joblib.dump(feature_cols, FEATURE_LIST_FILE)

print(f"✅ Đã lưu model tốt nhất tại: {os.path.abspath(MODEL_FILE)}")
print(f"✅ Đã lưu feature list tại: {os.path.abspath(FEATURE_LIST_FILE)}")
print("\n👉 BƯỚC TIẾP THEO: Chạy '05_evaluation.ipynb'.")